<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://gitlab.com/AkashNarayanan" target="_blank"> Akash Narayanan B</a></td>    
</table>
<hr>

So far, we have learnt about formulating combinatorial optimization problems as QUBO or Ising Model objective functions. We have also learnt how to convert between these two formulations. Our aim is to take advantage of Quantum Computers to solve these problems. To do so, we have to formulate our QUBO or Ising Model problems in a way that they can be run on a Quantum Computer.

The [Ocean SDK](https://github.com/dwavesystems/dwave-ocean-sdk) provides us many open-source tools to aid us in the problem solving process. Now let's take a look at the `BinaryQuadraticModel` class available in the `dimod` package of the Ocean SDK.

# Binary Quadratic Model (BQM)

`BinaryQuadraticModel` class helps us to encode our QUBO or Ising Model problems into a form suitable to be run on a Quantum Computer. Let us quickly recall the objective functions of QUBO and Ising Model.

The objective function of a QUBO is,

$$f(x) = \sum\limits_{i}^{N} {Q_{i, i} x_i} + \sum\limits_{i < j}^{N} {Q_{i, j} x_i x_j} \qquad\qquad x_i\in \{0,1\}$$

where the binary variables can take the values 0 and 1.

The objective function of an Ising Model is,

$$E(s) = \sum\limits_{i=1}^N h_i s_i + \sum\limits_{i<j}^N J_{i,j} s_i s_j   \qquad\qquad s_i \in\{-1,+1\}$$

where the binary variables can take the values -1 and +1 corresponding to the physical Ising spins.

The objective function of a Binary Quadratic Model is,

$$E(v) = \sum\limits_{i=1} a_i v_i + \sum\limits_{i<j} b_{i,j} v_i v_j + c \qquad\qquad v_i \in \{0,1\} \text{  or } \{-1,+1\}$$

We can notice that the binary variable $v_i$ can correspond either to $\{0, 1\}$ or to the physical Ising spins $\{-1, +1\}$. This way BQM can conveniently represent both a QUBO and an Ising Model.

# Creating an Instance of BQM

Let's first take a look at some of the essential parameters required to create an instance of the `BinaryQuadraticModel` class.

## Parameters

- `linear` 
   - The linear coefficients of the objective function should be defined as a dictionary.
   - The keys of the dictionary are the variables and their respective values are the linear coefficients associated with these variables. For example,

    ```python
    {'x1': 3, 'x2': 5, 'x3': 4, 'x4': 7}
    ```
    
- `quadratic`
   - The quadratic coefficients of the objective function should be defined as a dictionary.
   - The keys of the dictionary are the pairs of variables defined as tuples and their respective values are the quadratic coefficients associated with these pairs of variables. For example,

    ```python
    {('x1', 'x2'): 2, ('x2', 'x3'): 5}
    ```
                   
- `offset`
    - Constant energy offset value associated with the BQM can be set using this parameter. 
    - If there is no offset, it can be just set to `0`.
    
- `Vartype`
    - This parameter sets the variable type of the BQM. To create a QUBO instance, set this parameter as `'BINARY'`.
    - To create an Ising Model instance, set this parameter as `'SPIN'`.
    
## Example

Let us now try to create a BQM instance of the following objective function,

$$f(x_1, x_2, x_3, x_4) = - 5x_1 - 3x_2 - 8x_3 - 6x_4 + 4x_1 x_2 + 8x_1 x_3 + 2x_2 x_3 + 10x_3 x_4$$

We should define the linear and quadratic parts of the objective function as dictionaries and pass it as `linear` and `quadratic` parameters. In the objective function,

- The linear part is $- 5x_1 - 3x_2 - 8x_3 - 6x_4$. The corresponding dictionary can be defined as 

```python
{'x1': -5, 'x2': -3, 'x3': -8, 'x4': -6}
```
                       
- The quadratic part is $4x_1 x_2 + 8x_1 x_3 + 2x_2 x_3 + 10x_3 x_4$. The corresponding dictionary can be defined as

```python
{('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}
```  
- There is no offset, so we can set the `offset` parameter to `0`.

We can create a QUBO instance of BQM by setting the `Vartype` parameter to `'BINARY'`.

In [4]:
from dimod import BinaryQuadraticModel

linear = {'x1': -5, 'x2': -3, 'x3': -8, 'x4': -6}
quadratic = {('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}
offset = 0
vartype = 'BINARY'

bqm_qubo = BinaryQuadraticModel(linear, quadratic, offset, vartype)

We can create an Ising instance of BQM for the same objective function by setting the `Vartype` parameter to `'SPIN'`.

In [3]:
vartype = 'SPIN'

bqm_ising = BinaryQuadraticModel(linear, quadratic, offset, vartype)

## Attributes

We can check the values assigned to the parameters using the attributes of the BQM class.

```python
>>> bqm_qubo.linear
{'x1': -5.0, 'x2': -3.0, 'x3': -8.0, 'x4': -6.0}

>>> bqm_qubo.quadratic
{('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}

>>> bqm_qubo.offset
0

>>> bqm_qubo.vartype
<Vartype.BINARY: frozenset({0, 1})>

>>> bqm_ising.vartype
<Vartype.SPIN: frozenset({1, -1})>
```

These attributes are helpful to probe the details of an instance of BQM. The above two instances `bqm_qubo` and `bqm_ising` share the same attribute values except for the `vartype` attribute.

### Task 1

Create a QUBO instance of BQM for the following objective function

$$f(x_1, x_2) = 5x_1 + 7x_1 x_2 - 3x_2$$

[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-1)

### Task 2

Create an Ising instance of BQM for the following objective function

$$f(x_1, x_2, x_3, x_4) = x_1 + x_2 + x_3 + x_4 - 6x_1 x_3 - 6x_1 x_4 - 6x_3 x_4 - 6x_1 x_2$$

[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-2)

### Task 3

Find the `linear`, `quadratic`, `offset` and `vartype` values of the following instance of BQM.

In [12]:
%run bqm_functions.py

bqm_mystery = BinaryQuadraticModel(linear, quadratic, offset, vartype)

# Enter your code here





[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-3)

# References

1. ["Binary Quadratic Models"](https://test-projecttemplate-dimod.readthedocs.io/en/latest/reference/bqm/index.html), `dimod` Documentation, accessed August, 2021.
2. ["Getting Started"](https://docs.ocean.dwavesys.com/en/stable/getting_started.html#gs), D-Wave Ocean Software Documentation, accessed August, 2021.